In [1]:
import torch
from timm import create_model

# Load the raw state_dict
checkpoint = torch.load("student_model.pth", map_location=torch.device("cpu"))

# Adjust key names if they are stored under "model."
if "model" in checkpoint:
    checkpoint = checkpoint["model"]  # Extract the actual model state_dict

# Create model architecture
student_model = create_model("vit_tiny_patch16_224", pretrained=False, num_classes=10)

# Load adjusted state_dict
student_model.load_state_dict(checkpoint, strict=False)  # `strict=False` allows missing or extra keys

# Set model to evaluation mode
student_model.eval()


VisionTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (patch_drop): Identity()
  (norm_pre): Identity()
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=192, out_features=576, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=192, out_features=192, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (ls1): Identity()
      (drop_path1): Identity()
      (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=192, out_features=768, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()


In [2]:
checkpoint = torch.load("student_model.pth", map_location=torch.device("cpu"))
print(checkpoint.keys())


odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [3]:
checkpoint = {k.replace("model.", ""): v for k, v in checkpoint.items()}
student_model.load_state_dict(checkpoint, strict=False)


_IncompatibleKeys(missing_keys=['cls_token', 'pos_embed', 'patch_embed.proj.weight', 'patch_embed.proj.bias', 'blocks.0.norm1.weight', 'blocks.0.norm1.bias', 'blocks.0.attn.qkv.weight', 'blocks.0.attn.qkv.bias', 'blocks.0.attn.proj.weight', 'blocks.0.attn.proj.bias', 'blocks.0.norm2.weight', 'blocks.0.norm2.bias', 'blocks.0.mlp.fc1.weight', 'blocks.0.mlp.fc1.bias', 'blocks.0.mlp.fc2.weight', 'blocks.0.mlp.fc2.bias', 'blocks.1.norm1.weight', 'blocks.1.norm1.bias', 'blocks.1.attn.qkv.weight', 'blocks.1.attn.qkv.bias', 'blocks.1.attn.proj.weight', 'blocks.1.attn.proj.bias', 'blocks.1.norm2.weight', 'blocks.1.norm2.bias', 'blocks.1.mlp.fc1.weight', 'blocks.1.mlp.fc1.bias', 'blocks.1.mlp.fc2.weight', 'blocks.1.mlp.fc2.bias', 'blocks.2.norm1.weight', 'blocks.2.norm1.bias', 'blocks.2.attn.qkv.weight', 'blocks.2.attn.qkv.bias', 'blocks.2.attn.proj.weight', 'blocks.2.attn.proj.bias', 'blocks.2.norm2.weight', 'blocks.2.norm2.bias', 'blocks.2.mlp.fc1.weight', 'blocks.2.mlp.fc1.bias', 'blocks.2.ml

In [4]:
import torch

# Define example input (assuming image size is 224x224)
example_input = torch.randn(1, 3, 224, 224)

# Convert model to TorchScript
scripted_model = torch.jit.trace(student_model, example_input)

# Save the scripted model
torch.jit.save(scripted_model, "student_scripted.pt")

print("✅ Model successfully converted to TorchScript!")


c:\Users\user\Desktop\coffee_env\Lib\site-packages\torch\__init__.py:2132: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert condition, message


✅ Model successfully converted to TorchScript!


In [ ]:
import torch
import torch.nn.functional as F
from flask import Flask, request, jsonify
from PIL import Image
import io
import torchvision.transforms as transforms
import threading

# Initialize Flask app
app = Flask(__name__)

# Load the TorchScript model
model = torch.jit.load("student_scripted.pt", map_location="cpu")
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Define class labels
class_labels = ["longberry", "peaberry", "premium", "defect"]

# Home Route
@app.route("/")  
def home():
    return jsonify({"message": "Welcome to the Coffee Bean Quality Estimation API!"})

# Prediction Route
@app.route("/predict", methods=["POST"])
def predict():
    try:
        # Get the image from request
        file = request.files["file"]
        image = Image.open(io.BytesIO(file.read())).convert("RGB")

        # Preprocess the image
        image = transform(image).unsqueeze(0)  # Add batch dimension

        # Perform inference
        with torch.no_grad():
            output = model(image)
            probabilities = F.softmax(output, dim=1)
            predicted_class = torch.argmax(probabilities, dim=1).item()

        # Return the predicted label
        return jsonify({"prediction": class_labels[predicted_class], "confidence": probabilities[0][predicted_class].item()})

    except Exception as e:
        return jsonify({"error": str(e)})

# Run the Flask app
def run_app():
    app.run(host="0.0.0.0", port=5004)

thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5004
 * Running on http://10.10.134.78:5004
Press CTRL+C to quit
127.0.0.1 - - [27/Feb/2025 21:01:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 21:01:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [27/Feb/2025 21:14:57] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 21:16:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 21:16:30] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Feb/2025 21:18:10] "POST /predict HTTP/1.1" 200 -


In [ ]:
image_path = "C:/Users/user/Desktop/CSC YEAR 2/SEM 2/MACHINE LEARNING/dataset3/USK-Coffee/val/premium/90.jpg"
image = Image.open(image_path).convert("RGB")
image = transform(image).unsqueeze(0)

with torch.no_grad():
    output = model(image)
    print("Raw Model Output:", output)

    probabilities = F.softmax(output, dim=1)
    predicted_class = torch.argmax(probabilities, dim=1).item()

print("Predicted Class:", predicted_class)
print("Confidence:", probabilities[0][predicted_class].item())
print("Class Labels Order:", class_labels)


Raw Model Output: tensor([[ 0.2848, -0.2449,  0.0603, -0.4691, -0.3524, -0.3025,  0.0444,  0.1633,
          0.1960,  0.0096]])
Predicted Class: 0
Confidence: 0.13718801736831665
Class Labels Order: ['longberry', 'peaberry', 'premium', 'defect']
